In [1]:
!pip install -U google-genai pdfplumber python-docx gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.9/721.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.14.0
    Uninstalling gradio_client-1.14.0:
      Successfully uninstalled gradio_client-1.14.0
  Attempting uninstall: gr

In [2]:
import pdfplumber
import docx
from google import genai
from google.genai import types
import gradio as gr
from google.colab import userdata

# API Setup
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = "YOUR_GEMINI_API_KEY_HERE"

client = genai.Client(api_key=API_KEY)
MODEL_ID = "gemini-3-flash-preview"

# --- Text Extraction ---
def extract_text(file):
    if file is None: return ""
    try:
        file_path = file.name
        if file_path.endswith(".pdf"):
            text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    text += (page.extract_text() or "") + "\n"
            return text
        elif file_path.endswith(".docx"):
            doc = docx.Document(file_path)
            return "\n".join([p.text for p in doc.paragraphs])
    except Exception as e:
        return f"Error reading file: {str(e)}"
    return ""

# --- Feature 1: Question Generator ---
def generate_static_questions(file):
    if file is None: return "Please upload a resume first."
    resume_text = extract_text(file)
    prompt = f"Analyze this resume and give 5 technical and 3 HR questions: {resume_text}"
    try:
        response = client.models.generate_content(model=MODEL_ID, contents=prompt)
        return response.text
    except Exception as e:
        return f"API Error: {str(e)}"

# --- Feature 2: Interview Simulator (Gradio 5+ Compatible) ---
# রেজ্যুমে টেক্সট ধরে রাখার জন্য একটি গ্লোবাল ভেরিয়েবল
resume_cache = {"text": "", "file_name": ""}

def chat_with_interviewer(message, history, file):
    if file is None:
        return "Please upload your resume in the 'Upload Resume' box below to start."

    # বারবার ফাইল রিড না করে ক্যাশ থেকে চেক করা (এটি চ্যাটকে ফাস্ট করবে)
    if resume_cache["file_name"] != file.name:
        resume_cache["text"] = extract_text(file)
        resume_cache["file_name"] = file.name

    resume_text = resume_cache["text"]

    system_instr = f"""
    You are an expert AI Interviewer.
    Candidate Resume: {resume_text}

    CRITICAL RULES:
    1. Always acknowledge and give a brief feedback on the user's specific answer before asking anything new.
    2. Ask ONLY one question at a time.
    3. Do not jump to a new topic if the user is still answering a previous question.
    4. Maintain a conversational flow like a real human interviewer.
    """

    # Gradio 6+ History Mapping (নিখুঁত চ্যাটের জন্য)
    messages = []
    for turn in history:
        # User role handling
        u_msg = turn['content'] if isinstance(turn, dict) and turn['role'] == 'user' else (turn[0] if isinstance(turn, (list, tuple)) else "")
        # Assistant role handling
        a_msg = turn['content'] if isinstance(turn, dict) and turn['role'] == 'assistant' else (turn[1] if isinstance(turn, (list, tuple)) else "")

        if u_msg:
            messages.append({"role": "user", "parts": [{"text": str(u_msg)}]})
        if a_msg:
            messages.append({"role": "model", "parts": [{"text": str(a_msg)}]})

    # বর্তমান মেসেজ যোগ করা
    messages.append({"role": "user", "parts": [{"text": str(message)}]})

    try:
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=messages,
            config=types.GenerateContentConfig(
                system_instruction=system_instr,
                temperature=0.7 # কিছুটা ক্রিয়েটিভ উত্তরের জন্য
            )
        )
        return response.text
    except Exception as e:
        print(f"Debug Error: {e}")
        return "I had a momentary glitch. Could you please repeat that?"

# --- Integrated UI ---
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 Smart Resume Pro: Interview Coach & Simulator")

    with gr.Tab("📋 Question Generator"):
        input_file = gr.File(label="Upload Resume (PDF/DOCX)")
        gen_btn = gr.Button("Generate Questions")
        output_txt = gr.Textbox(label="Tailored Questions", lines=10)
        gen_btn.click(generate_static_questions, inputs=[input_file], outputs=[output_txt])

    with gr.Tab("💬 Live Interview Simulator"):
        chat_file = gr.File(label="Upload Resume (PDF/DOCX required for chat)")
        gr.ChatInterface(
            fn=chat_with_interviewer,
            additional_inputs=[chat_file],
            examples=[["Hi, start my interview!"], ["What do you think of my projects?"]]
        )

if __name__ == "__main__":
    # Colab এ 'share=True' দিতে ভুলবেন না
    demo.launch(share=True, theme=gr.themes.Soft())

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc8a27c38229353a45.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
